In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#importing necessary libraries
import os 
import re
import json
import pathlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
import cv2 
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
with open('/kaggle/input/classification-new/classification_new.json', 'r') as json_file:
    labels_data = json.load(json_file)

2024-03-02 03:56:37.298502: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-02 03:56:37.298611: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-02 03:56:37.414742: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
##Storing labels in array Y
# Storing images in X
img_size=50
X=[]
Y=[]
j=1
for dirname, _, filenames in os.walk('/kaggle/input/biofer-images-new/biofors_images/biofors_images'):
    if(j):
        j=0
        continue
    num=re.findall(r'\d+$', dirname)[-1]
#     print(num)
    for i in filenames:
        k=os.path.join(dirname, i)
        img=cv2.imread(k,cv2.IMREAD_COLOR)
        img=cv2.resize(img,(img_size,img_size))
        X.append(np.array(img))
        try:
            Y.append(labels_data[str(num)][str(i)])
        except KeyError:
            Y.append("Non-scientific") 
        #Y.append(labels_data[str(num)][str(i)])
        #print(labels_data[str(num)][str(i)])
print(len(X))

55005


In [4]:
#Encoding Y using label encoder and mapping class number to class name
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
X = np.array(X)
Y = np.array(Y)

# Encode the labels
le = LabelEncoder()
Y_encoded = le.fit_transform(Y)
class_mapping = {number: label for label, number in zip(le.classes_, le.transform(le.classes_))}
print("Class Number to Label Mapping:")
for number, label in class_mapping.items():
    print(f"{number}: {label}")
Y_encoded = to_categorical(Y_encoded)


Class Number to Label Mapping:
0: Blot/Gel
1: FACS
2: Macroscopy
3: Microscopy
4: Non-scientific


In [5]:
#splitting in training testing
x_train, x_test, y_train, y_test = train_test_split(X, Y_encoded, test_size=0.25, random_state=5)

In [6]:
#creating VGG model
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [7]:
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(len(le.classes_), activation='softmax')(x)

# Creating the final model
model = Model(inputs=base_model.input, outputs=predictions)

In [8]:
# Freezing the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Compiling the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.save('vgg_model.h5')


In [9]:

model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=32)


loss, accuracy = model.evaluate(x_test, y_test)
print("Loss:", loss)
print("Accuracy:", accuracy)

Epoch 1/10
  18/1290 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.5781 - loss: 8.1493 

I0000 00:00:1709352071.420304     105 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1290/1290 ━━━━━━━━━━━━━━━━━━━━ 27s 16ms/step - accuracy: 0.9216 - loss: 1.2345 - val_accuracy: 0.9669 - val_loss: 0.1738
Epoch 2/10
1290/1290 ━━━━━━━━━━━━━━━━━━━━ 15s 12ms/step - accuracy: 0.9622 - loss: 0.2155 - val_accuracy: 0.9767 - val_loss: 0.1323
Epoch 3/10
1290/1290 ━━━━━━━━━━━━━━━━━━━━ 15s 12ms/step - accuracy: 0.9674 - loss: 0.1942 - val_accuracy: 0.9767 - val_loss: 0.1392
Epoch 4/10
1290/1290 ━━━━━━━━━━━━━━━━━━━━ 15s 12ms/step - accuracy: 0.9693 - loss: 0.1951 - val_accuracy: 0.9751 - val_loss: 0.1444
Epoch 5/10
1290/1290 ━━━━━━━━━━━━━━━━━━━━ 15s 12ms/step - accuracy: 0.9748 - loss: 0.1976 - val_accuracy: 0.9793 - val_loss: 0.1127
Epoch 6/10
1290/1290 ━━━━━━━━━━━━━━━━━━━━ 15s 12ms/step - accuracy: 0.9725 - loss: 0.1773 - val_accuracy: 0.9810 - val_loss: 0.1087
Epoch 7/10
1290/1290 ━━━━━━━━━━━━━━━━━━━━ 15s 12ms/step - accuracy: 0.9760 - loss: 0.1437 - val_accuracy: 0.9807 - val_loss: 0.1163
Epoch 8/10
1290/1290 ━━━━━━━━━━━━━━━━━━━━ 15s 12ms/step - accuracy: 0.9796 - loss: 0.09

In [10]:
#number of images sampled in each class
import numpy as np

y_train_indices = np.argmax(y_train, axis=1)

# Counting the number of images in each class
class_counts = {class_idx: len(y_train_indices[y_train_indices == class_idx]) for class_idx in range(len(le.classes_))}

for class_idx, count in class_counts.items():
    print(f"Class {class_idx}: {count} images")

Class 0: 20614 images
Class 1: 797 images
Class 2: 914 images
Class 3: 13553 images
Class 4: 5375 images


In [11]:
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

y_test_indices = np.argmax(y_test, axis=1)

y_pred = model.predict(x_test)
y_pred_class = np.argmax(y_pred, axis=1)

cm = confusion_matrix(y_test_indices, y_pred_class)


plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=le.classes_, yticklabels=le.classes_)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


report = classification_report(y_test_indices, y_pred_class, target_names=le.classes_)
print("Classification Report:")
print(report)


430/430 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step


Classification Report:
                precision    recall  f1-score   support

      Blot/Gel       0.99      0.99      0.99      6826
          FACS       0.94      0.93      0.94       264
    Macroscopy       0.87      0.74      0.80       280
    Microscopy       0.98      0.98      0.98      4557
Non-scientific       1.00      1.00      1.00      1825

      accuracy                           0.98     13752
     macro avg       0.95      0.93      0.94     13752
  weighted avg       0.98      0.98      0.98     13752



In [13]:
from sklearn.metrics import precision_recall_fscore_support

precision, recall, f1_score, support = precision_recall_fscore_support(y_test_indices, y_pred_class, average='weighted')

print("F1 Score:", f1_score)

F1 Score: 0.9812251304082887


In [14]:
from tensorflow.keras.models import load_model
import cv2
import numpy as np

# Loading the previously trained model
model = load_model("/kaggle/working/vgg_model.h5")
img_width, img_height =50, 50
img_path ="/kaggle/input/image-1/obj1__5.png"
img = cv2.imread(img_path)
if img is None:
    print("Error: Could not read image at", img_path)
img = cv2.resize(img,(50,50))  
img = np.expand_dims(img, axis=0)  

img = img / 255.0
predictions = model.predict(img)

predicted_class = np.argmax(predictions)

print("Predicted class:", predicted_class)
predicted_class_label = class_mapping.get(predicted_class, "Unknown")
print("Predicted Class Label:", predicted_class_label)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted class: 4
Predicted Class Label: Non-scientific


In [18]:
#RESNET
import os
import cv2
import numpy as np
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split


img_size = 50

base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [20]:
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(5, activation='softmax')(x)  

# Creating the final model
model = Model(inputs=base_model.input, outputs=predictions)

In [21]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [22]:
X = np.array(X)
Y = np.array(Y)
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

le = LabelEncoder()
Y_encoded = le.fit_transform(Y)
Y_onehot = to_categorical(Y_encoded)

x_train, x_test, y_train, y_test = train_test_split(X, Y_onehot, test_size=0.2, random_state=42)

In [23]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.save('resnet_model.h5')

In [24]:

model.fit(x_train, y_train, epochs=7, batch_size=32, validation_data=(x_test, y_test))


loss, accuracy = model.evaluate(x_test, y_test)
print("Validation Loss:", loss)
print("Validation Accuracy:", accuracy)

Epoch 1/7
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.9355 - loss: 0.3038

W0000 00:00:1709353790.183757     103 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1376/1376 ━━━━━━━━━━━━━━━━━━━━ 142s 61ms/step - accuracy: 0.9355 - loss: 0.3036 - val_accuracy: 0.9727 - val_loss: 0.0878
Epoch 2/7
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 53s 38ms/step - accuracy: 0.9752 - loss: 0.0837 - val_accuracy: 0.9454 - val_loss: 0.2057
Epoch 3/7
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 52s 38ms/step - accuracy: 0.9796 - loss: 0.0670 - val_accuracy: 0.8972 - val_loss: 0.3481
Epoch 4/7
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 53s 38ms/step - accuracy: 0.9804 - loss: 0.0604 - val_accuracy: 0.9645 - val_loss: 0.1183
Epoch 5/7
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 53s 38ms/step - accuracy: 0.9755 - loss: 0.0838 - val_accuracy: 0.9706 - val_loss: 0.0878
Epoch 6/7
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 53s 38ms/step - accuracy: 0.9846 - loss: 0.0533 - val_accuracy: 0.9229 - val_loss: 0.6970
Epoch 7/7
1376/1376 ━━━━━━━━━━━━━━━━━━━━ 54s 39ms/step - accuracy: 0.9497 - loss: 0.1915 - val_accuracy: 0.9735 - val_loss: 0.0852
344/344 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.9720 - loss: 0.0863
Validation Loss:

In [25]:
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns


y_test_indices = np.argmax(y_test, axis=1)

y_pred = model.predict(x_test)
y_pred_class = np.argmax(y_pred, axis=1)

cm = confusion_matrix(y_test_indices, y_pred_class)

# Plotting the confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=le.classes_, yticklabels=le.classes_)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# Generating the classification report
report = classification_report(y_test_indices, y_pred_class, target_names=le.classes_)
print("Classification Report:")
print(report)


344/344 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step


Classification Report:
                precision    recall  f1-score   support

      Blot/Gel       0.98      0.99      0.98      5513
          FACS       0.92      0.98      0.94       210
    Macroscopy       0.92      0.51      0.66       263
    Microscopy       0.96      0.98      0.97      3547
Non-scientific       1.00      1.00      1.00      1468

      accuracy                           0.97     11001
     macro avg       0.96      0.89      0.91     11001
  weighted avg       0.97      0.97      0.97     11001



In [26]:
from sklearn.metrics import precision_recall_fscore_support

precision, recall, f1_score, support = precision_recall_fscore_support(y_test_indices, y_pred_class, average='weighted')

# Print the F1 score
print("F1 Score:", f1_score)

F1 Score: 0.9717972465981277


In [31]:
from tensorflow.keras.models import load_model
import cv2
import numpy as np

model = load_model("/kaggle/working/resnet_model.h5")
img_width, img_height =50, 50
img_path ="/kaggle/input/imagepredict2/image_3.png"
img = cv2.imread(img_path)
if img is None:
    print("Error: Could not read image at", img_path)
img = cv2.resize(img,(50,50)) 
img = np.expand_dims(img, axis=0)  

img = img / 255.0

predictions = model.predict(img)

predicted_class = np.argmax(predictions)

print("Predicted class:", predicted_class)
predicted_class_label = class_mapping.get(predicted_class, "Unknown")
print("Predicted Class Label:", predicted_class_label)

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Predicted class: 3
Predicted Class Label: Microscopy
